# This notebook formats the bulk RNAseq expression data to match the metadata fields from the generatedpseudobulks

Intended inputs to this dataset are paired `.tsv` expression and metadata files

The end product of this notebook will a single `.h5ad` file that includes all bulk obervations and the following metadata fields:
- stimulation column `stim`
- sample id column `sample_id`

In [1]:
import sys
import pathlib
import yaml
import subprocess
import pickle

import pandas as pd
import anndata as ad

## Load config
The config file specifies the path to data and software repo (due to currently in active development)

In [2]:
# Get the root directory of the analysis repository
REPO_ROOT = subprocess.run(
    ["git", "rev-parse", "--show-toplevel"], capture_output=True, text=True
).stdout.strip()
REPO_ROOT = pathlib.Path(REPO_ROOT)

CONFIG_FILE = REPO_ROOT / 'config.yml'
assert CONFIG_FILE.exists(), f"Config file not found at {CONFIG_FILE}"

with open(CONFIG_FILE, 'r') as file:
    config_dict = yaml.safe_load(file)
ACCESSION = config_dict['data_accession']

## Parameters

In [3]:
analysis_config = config_dict['analysis_config']
source_col_def = analysis_config['source']['bulk']
format_col_def = analysis_config['format']

SAMPLE_ID_COL = format_col_def['obs']['sample_col']
STIM_COL = format_col_def['obs']['stim_col']
GENE_ID_COL = format_col_def['var']['gene_id_col']
DATASPLIT_SEED = 42

## Define Path to write Pre-Processing Outputs

In [4]:
PREPROCESSING_OUTPUT_PATH = REPO_ROOT / 'processed_data'
assert PREPROCESSING_OUTPUT_PATH.exists(), f"Preprocessing output path {PREPROCESSING_OUTPUT_PATH} does not exist"
BULK_FORMAT_DATA_PATH = PREPROCESSING_OUTPUT_PATH / 'bulk_formatted'
BULK_FORMAT_DATA_PATH.mkdir(exist_ok=True, parents=True)
BULK_FORMAT_EXPR_FILE = BULK_FORMAT_DATA_PATH / f'{ACCESSION}bulk_processed.h5ad'

## Preprocessing of Bulk Data

## Load anndata object and export

In [5]:
adata = ad.read_h5ad(
    REPO_ROOT / 'processed_data' / f'{ACCESSION}_qc_norm.h5ad'
)

In [ ]:
adata.var[GENE_ID_COL] = adata.var[
    source_col_def['var']['gene_id_col']]
adata.obs[SAMPLE_ID_COL] = adata.obs[
    source_col_def['obs']['sample_col']].tolist()
adata.obs[STIM_COL] = adata.obs[
    source_col_def['obs']['stim_col']]

for col in adata.obs.columns:
    if adata.obs[col].dtype == object or adata.obs[col].dtype == "category":
        adata.obs[col] = adata.obs[col].astype("str")

adata.write(BULK_FORMAT_EXPR_FILE)

In [7]:
ct = pd.crosstab(adata.obs[SAMPLE_ID_COL], adata.obs[STIM_COL])
with pd.option_context(
    'display.max_rows', None,
    'display.max_columns', None,
    'display.width', None,
    'display.max_colwidth', None
):
    print(ct)

stim       chunk  dissociated
sample_id                    
2251           1            2
2267           1            2
2283           1            2
2293           1            2
2380           1            2
2428           1            2
2467           1            2
2497           1            2


In [8]:
gene_out_file = BULK_FORMAT_DATA_PATH / f'{ACCESSION}_genes.pkl'
gene_ids = adata.var[GENE_ID_COL]
pickle.dump(gene_ids, open( gene_out_file, "wb" ) )